In [45]:
import psycopg2
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tqdm import tqdm
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [94]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

#데이터 가져오기
cur.execute("select distinct title, date, url, hashtag from yt_clothes where title like '%관리%'")
conn.commit()
yy = cur.fetchall()

#DF화
yyy = pd.DataFrame(yy)
yyy.columns = ['title', 'date', 'url', 'hashtag']

#db 연결 종료
conn.cursor().close()
conn.close()

In [96]:
yyy.head()

,title,date,url,hashtag
0,#탈코르셋 ?! 아름다운 허리라인 만들어주는 #위뷰티 #코르셋피부관리 !,2022-05-20,https://www.youtube.com/watch?v=chcQ-JheJBo,#코르셋 #코르셋관리 #코르셋피부관리 #개미허리 #허리라인 #허리라인만드는법 #위뷰...
1,[ENG] Vlog 먹을거 다 먹으면서 관리하는 브이로그🥘,2021-10-02,https://www.youtube.com/watch?v=z_uYedo6Hs8,#관리의날브이로그 #일상브이로그 #다이어트 #워너랩 #워너버닝다이어트
2,"[ENG] 등드름 흉터 bye 꼭 봐주세요,,강추 2022 여름준비 관리의날👙🐠 ...",2022-05-08,https://www.youtube.com/watch?v=8xb9PN5rCEM,#관리의날 #홈케어 #알라딘필링후기 #셀프염색약추천
3,[eng] 밀라논나 '최후의 신발'을 공개합니다. (+신발 오래 신는 관리팁),2021-08-25,https://www.youtube.com/watch?v=fxSQMVeP8hM,
4,"[ENG] 찐친 4인방 왓츠인마이백 + 꿀템추천👛 (ft.웹디,피부관리사,냥집사) ...",2021-08-23,https://www.youtube.com/watch?v=W4FR9f5ivBE,#왓츠인마이백 #가방공개 #꿀템추천


In [95]:
yyy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    42 non-null     object
 1   date     42 non-null     object
 2   url      42 non-null     object
 3   hashtag  42 non-null     object
dtypes: object(4)
memory usage: 1.4+ KB


In [97]:
# 모든 타이틀 제목을 연결해서 하나의 텍스트 문서로 만들려 함
titles = yyy['title'].tolist()
titles[:5]

['#탈코르셋 ?! 아름다운 허리라인 만들어주는 #위뷰티 #코르셋피부관리 !',
 '[ENG] Vlog 먹을거 다 먹으면서 관리하는 브이로그🥘',
 '[ENG] 등드름  흉터 bye 꼭 봐주세요,,강추 2022 여름준비 관리의날👙🐠 (셀프 알라딘필링, 올리브영 꿀템)',
 "[eng] 밀라논나 '최후의 신발'을 공개합니다. (+신발 오래 신는 관리팁)",
 "[ENG] 찐친 4인방 왓츠인마이백 + 꿀템추천👛 (ft.웹디,피부관리사,냥집사) l What's In My Bag?"]

In [98]:
okt = Okt()

def rmEmoji(inputData):

    emoji_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
            "]+", flags=re.UNICODE)

    return emoji_pattern.sub(r'', inputData) # no emoji

#불용어 읽어오기
stopwords = pd.read_csv("stopwords.txt")
stopwords = stopwords['word'].tolist()
stopwords.extend([',', '.', '+', '[', ']', '!', '?', '(', ')', '|', '_', '~', '#', '/'])

#불용어제거 + 토큰화
tks = []
for sentence in tqdm(titles) :
    sentence = rmEmoji(sentence) # 이모티콘 제거
    tokenized_sentence = okt.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tks.append(stopwords_removed_sentence)

100%|██████████| 42/42 [00:00<00:00, 660.39it/s]


In [99]:
clean_sentences = []
for tk in tks :
    clean_sentences.append(" ".join(tk))

clean_sentences

['#탈코르셋 ?! 아름다운 허리 라인 만들어주는 #위뷰티 #코르셋피부관리',
 'ENG Vlog 먹을거 다 먹으면서 관리 하는 브이 로그',
 'ENG 등드름 흉터 bye 꼭 봐주세요 ,, 강추 2022 여름 준비 관리 날 셀프 알라딘 필링 올리브영 꿀템',
 "eng 밀 라 논 ' 최후 신발 ' 공개 합니다 (+ 신발 오래 신는 관리 팁",
 "ENG 찐 친 4 인방 왓츠 인 마이 백 꿀템 추천 ft 웹디 피부 관리사 냥 집사 l What ' s In My Bag",
 'ENG 한 달 야식 끊고 (?) 몸 관리 하는 일상 브이 로그 ft GRN 클린 샾',
 'vlog 3 화장 법공 개 쇼핑몰 세금 관리 방법 배 대표 출현 ㅋㅋ 꽁냥꽁 냥 브이 로그',
 '뉴스 zip 겨울철 필수품 롱 패딩 관리 법 ZIP MBC 뉴스 #뉴스zip',
 '옷 잘 입는 법 여자 린넨 구매 하기 전 꼭 알아야 할 꿀팁 총 정리 ft 세탁 법 관리 법',
 '※ 칼 주의 ※ 최은경 냉장고 정리 법 매일 쓰는 식 재료 TMI 알려 드림 ⚡️ 최은경 관리 사무소',
 '100만원 어치 프리 쥬 붙임머리 드디어 도 해봤어요 !! 아이돌 다니는 샵 은 역시 달라 ft 붙임머리 3년 차 찐 후기 관리 법 장단점 붙임머리 고데기',
 '2022년 인생 바뀌는 남자 관리 5 가지',
 'How 시리즈 체육관 팀 조끼 효율 적 관리 하는 방법',
 'SPA 브랜드 여름 신상 ‼️ 구경 하러 가는 날 어떤게 나왔을까 퍼스 널 컬러 GRWM 여름철 피부관리 비법',
 'vlog 데 일리 룩 던스트 코트 트 위드 자켓 보테가베네타 카세트 백 미니 l 하객 룩 lCOS 자라 H & Ml 코 무 포이 앤 더스토리 즈 시야 쥬 토즈 퍼 l 두피 관리 닥터 포 헤어 ), 호캉스 l 겨울 코디',
 '짧은 머리 손질 법 피부관리 【 일상 】',
 '여름 원피스 고민 끝 TPO 맞는 원피스 6 종 추천 최은경 관리 사무소',
 '아우 터 안 뭐 입지 1만 원대 시작 하는 겨울 ☃️❄️ 최은경 관리 사무소',

In [100]:
# title수 * title수 의 TF-IDF 상관행렬 만들기
n = len(clean_sentences)
text_mat = np.zeros(shape=(n,n), dtype=np.float64)

#TF-IDF
tfidf = TfidfVectorizer(analyzer='word', encoding='utf-8')
tfidf.fit(clean_sentences)

TfidfVectorizer()

In [101]:
tfidf.vocabulary_

{'탈코르셋': 250,
 '아름다운': 168,
 '허리': 280,
 '라인': 100,
 '만들어주는': 109,
 '위뷰티': 201,
 '코르셋피부관리': 247,
 'eng': 15,
 'vlog': 33,
 '먹을거': 119,
 '먹으면서': 118,
 '관리': 55,
 '하는': 270,
 '브이': 134,
 '로그': 102,
 '등드름': 95,
 '흉터': 288,
 'bye': 10,
 '봐주세요': 131,
 '강추': 44,
 '2022': 4,
 '여름': 185,
 '준비': 225,
 '셀프': 150,
 '알라딘': 171,
 '필링': 265,
 '올리브영': 193,
 '꿀템': 68,
 '최후': 238,
 '신발': 164,
 '공개': 54,
 '합니다': 273,
 '오래': 190,
 '신는': 163,
 '인방': 204,
 '왓츠': 194,
 '마이': 108,
 '추천': 239,
 'ft': 17,
 '웹디': 199,
 '피부': 263,
 '관리사': 56,
 '집사': 229,
 'what': 34,
 'in': 21,
 'my': 29,
 'bag': 9,
 '야식': 177,
 '끊고': 70,
 '일상': 207,
 'grn': 18,
 '클린': 249,
 '화장': 285,
 '법공': 127,
 '쇼핑몰': 156,
 '세금': 145,
 '방법': 126,
 '대표': 84,
 '출현': 240,
 'ㅋㅋ': 39,
 '꽁냥꽁': 67,
 '뉴스': 77,
 'zip': 37,
 '겨울철': 48,
 '필수품': 267,
 '패딩': 257,
 'mbc': 26,
 '뉴스zip': 78,
 '입는': 209,
 '여자': 188,
 '린넨': 107,
 '구매': 60,
 '하기': 269,
 '알아야': 173,
 '꿀팁': 69,
 '정리': 217,
 '세탁': 148,
 '주의': 224,
 '최은경': 237,
 '냉장고': 76,
 '매일': 115,
 '쓰는': 166,


In [102]:
#문장의 모든 순서쌍마다 vector 간의 cosine similarity를 구함
from sklearn.metrics.pairwise import cosine_similarity

for idxi, i in enumerate(clean_sentences) :
    for idxj, j in enumerate(clean_sentences) :
        # row, col이 같으면 스킵
        if idxi == idxj:
            continue
        
        text_mat[idxi][idxj] = cosine_similarity(tfidf.transform([i]), tfidf.transform([j]))[0][0]

text_mat

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.07434377, ..., 0.0073214 , 0.00875343,
        0.01093706],
       [0.        , 0.07434377, 0.        , ..., 0.0046764 , 0.00559108,
        0.00698582],
       ...,
       [0.        , 0.0073214 , 0.0046764 , ..., 0.        , 0.00458034,
        0.00572295],
       [0.        , 0.00875343, 0.00559108, ..., 0.00458034, 0.        ,
        0.00684233],
       [0.        , 0.01093706, 0.00698582, ..., 0.00572295, 0.00684233,
        0.        ]])

In [103]:
# 인접리스트 방식으로 노드 간의 연결관계 표현

mat = text_mat.copy()
output_node = []

for row in mat:
    tmp=[]
    for i in range(len(row) ):
        if row[i]>0 :
            tmp.append(i)
    output_node.append(tmp)


for i in range(len(output_node)) :
    print(len(output_node[i]), end=' ')

0 38 39 38 12 39 37 37 38 37 38 37 37 12 37 2 39 37 37 37 38 37 37 37 39 37 38 38 38 37 37 37 37 38 39 38 38 38 38 37 37 38 

In [104]:
clean_sentences[0]

'#탈코르셋 ?! 아름다운 허리 라인 만들어주는 #위뷰티 #코르셋피부관리'

In [109]:
clean_sentences[14]

'vlog 데 일리 룩 던스트 코트 트 위드 자켓 보테가베네타 카세트 백 미니 l 하객 룩 lCOS 자라 H & Ml 코 무 포이 앤 더스토리 즈 시야 쥬 토즈 퍼 l 두피 관리 닥터 포 헤어 ), 호캉스 l 겨울 코디'

In [110]:
d = 0.85
initial_tr=1/len(mat)
TR = [initial_tr]*len(mat)

TR

[0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808,
 0.023809523809523808]

In [ ]:
#while문을 탈출하기 위한 boolean 변수
stop = False